<a href="https://colab.research.google.com/github/m-manuelmussa/Machine-Learning-QSAR-2D-TKI-HER2-2025/blob/main/2_Pr%C3%A9_processamento_de_dados_%26_C%C3%A1lculo_de_Features_de_Lipinski_%26_PubChemFP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***Desenvolvimento de modelos de machine learning baseados em QSAR-2D para a predição de novos candidatos a fármacos TKI-HER2 para o tratamento de câncer da mama***

---



*@Micliete_Mussa*

### ***2. Machine Learning & QSAR-2D***

### ***2.1. Pré-processamento de dados***

### **Pipeline**
1. Configuração do ambiente de trabalho
2. Carregamento dos dados em .csv
3. Conversão de SMILES em Mol
4. Remoção de sais
5. Padronização de tautômeros
6. Remoção de moléculas não Kekulizadas
7. Remoção de duplicatas
8. Conversão de IC50 em pIC50
9. Binarização de pIC50 em activo/Inactivo

### ***1. Configuração do ambiente de trabalho***

In [ ]:
#Instalação de frameworks necessários
!pip install rdkit #Manipulação de dados de compostos químicos

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.2/36.2 MB 30.3 MB/s eta 0:00:00


In [ ]:
#Instalação de frameworks necessários
import pandas as pd #Tabulação de dados
import numpy as np #Operações matemáticas
from rdkit import Chem #Manipulação e curadoria de dados de compostos químicos
from rdkit.Chem import SaltRemover, MolStandardize #Remoção de sais dos compostos
from rdkit.Chem.MolStandardize import rdMolStandardize #Padronização dos compostos (normalização de tautômeros)
from rdkit.Chem import Descriptors, Lipinski, AllChem # Calcular Features de Lipinski
from rdkit.Chem import rdMolDescriptors # Calcular Fingerprints de PubChem

### ***2. Carregamento dos dados em .csv***

In [ ]:
df = pd.read_csv("Dataset_final.csv", sep=";")  # Deve ter colunas 'SMILES' e 'IC50' para pré-processar os compostos

### ***3. Conversão de SMILES em Mol***

In [ ]:
df['Mol'] = df['SMILES'].apply(lambda x: Chem.MolFromSmiles(x))
df = df[df['Mol'].notnull()].reset_index(drop=True) #Permite converter os SMILES em arquivo manipulável sob a dimensão química

### ***4. Remoção de sais***

In [ ]:
remover = SaltRemover.SaltRemover()
def remover_sais(mol): # Estrututa de uma função que permite remover sais em todos os compostos
    try:
        mol = remover.StripMol(mol, dontRemoveEverything=True)
        return mol
    except:
        return None
df['Mol'] = df['Mol'].apply(remover_sais) # Aplicação da função de remoção de sais
df = df[df['Mol'].notnull()].reset_index(drop=True)

### ***5. Padronização de tautômeros***

In [ ]:
tautomer_enumerator = rdMolStandardize.TautomerEnumerator()
df['Mol'] = df['Mol'].apply(lambda mol: tautomer_enumerator.Canonicalize(mol))
# Trata conjunto de compostos isómeros transformando em único composto, para evitar duplicação artificial

[08:49:20] Tautomer enumeration stopped at 171 tautomers: max transforms reached
[08:49:21] Tautomer enumeration stopped at 171 tautomers: max transforms reached
[08:49:22] Tautomer enumeration stopped at 171 tautomers: max transforms reached
[08:49:23] Tautomer enumeration stopped at 156 tautomers: max transforms reached
[08:49:24] Tautomer enumeration stopped at 165 tautomers: max transforms reached
[08:49:24] Tautomer enumeration stopped at 171 tautomers: max transforms reached
[08:49:25] Tautomer enumeration stopped at 171 tautomers: max transforms reached
[08:49:26] Tautomer enumeration stopped at 171 tautomers: max transforms reached
[08:49:27] Tautomer enumeration stopped at 183 tautomers: max transforms reached
[08:49:28] Tautomer enumeration stopped at 195 tautomers: max transforms reached
[08:49:28] Tautomer enumeration stopped at 171 tautomers: max transforms reached
[08:49:29] Tautomer enumeration stopped at 156 tautomers: max transforms reached
[08:49:38] Can't kekulize mo

### ***6. Remoção de moléculas não Kekulizadas***

In [ ]:
def mol_valido(mol):
    try:
        Chem.Kekulize(mol, clearAromaticFlags=True)
        return True
    except:
        return False

df['Kekuliza'] = df['Mol'].apply(mol_valido)
df = df[df['Kekuliza']].reset_index(drop=True)
df.drop(columns=['Kekuliza'], inplace=True)
# Remove todos compostos com o sistema aromático anômalo e moléculas com a estrutura central incompleta

### ***7. Remoção de duplicatas***

In [ ]:
df['Canonical_SMILES'] = df['Mol'].apply(lambda mol: Chem.MolToSmiles(mol))
df = df.drop_duplicates(subset='Canonical_SMILES').reset_index(drop=True)
# Remove duplicatas geradas após a remoção de sais

### ***8. Conversão de IC50 em pIC50***

In [ ]:
# IC50 deve estar em nM (nanomolar)
def converter_para_pIC50(ic50_nM):
    try:
        if ic50_nM > 0:
            return -np.log10(ic50_nM * 1e-9)
        else:
            return np.nan
    except:
        return np.nan

df['pIC50'] = df['IC50_nM'].apply(converter_para_pIC50)
df = df[df['pIC50'].notnull()].reset_index(drop=True)
# Permite converter o IC50 em escala logarítmica para garantir o controle de múltiplas escalas de IC50 em uma logarítmica

### ***9. Categorização de valores de pIC50***

In [ ]:
# Definir limiar (pIC50 >= 6.0 → ativo (1); senão → inativo (0))
limiar = 6.0

# Binarizar em 0 e 1
df['Classe'] = df['pIC50'].apply(lambda x: 1 if x >= limiar else 0)

### ***10. Dados pré-processados***

In [ ]:
df['Name'] = df.index.astype(str)
df_final = df[['Canonical_SMILES', 'pIC50', 'Classe']]
display(df_final.head())

,Canonical_SMILES,pIC50,Classe
0,C=C1C=C(C)NC1=CC1=C(O)NC2=NC=NC(NC3=CC=C(F)C(C...,6.522879,1
1,C=C1C=C(C(=O)N2CCOCC2)NC1=CC1=C(O)NC2=NC=NC(NC...,6.397940,1
2,C=C1C=C(C(=O)N2CCOCC2)NC1=CC1=C(O)NC2=NC=NC(NC...,7.000000,1
3,C=C1C=C(C(=O)N2CCOCC2)NC1=CC1=C(O)NC2=NC=NC(NC...,5.301030,0
4,C=C1C=C(C(=O)NCCN2CCOCC2)NC1=CC1=C(O)NC2=NC=NC...,7.000000,1


### ***2.2. Cálculo de descritores de Lipinski e PubChem Fingerprints***

## **Pipeline**
1. Seleccionar descritores de Lipinski e Fingerprints de PubChem
2. Calcular descritores de Lipinski e PubChemFP para todos SMILES
3. Concatenar com dados pré-processados
4. Salvar os dados para EDA

### ***1. Seleccionar descritores de Lipinski e Fingerprints de PubChem***

In [ ]:
def calcular_descritores_fp(smiles):
    mol = Chem.MolFromSmiles(smiles)

    # === Descritores de Lipinski ===
    mol_logp = Descriptors.MolLogP(mol)
    mol_wt = Descriptors.MolWt(mol)
    hbd = Lipinski.NumHDonors(mol)
    hba = Lipinski.NumHAcceptors(mol)

    # === Fingerprint tipo PubChem ===
    fp = Chem.RDKFingerprint(mol, fpSize=881)
    fp_bits = list(fp)

    return [mol_logp, mol_wt, hbd, hba] + fp_bits
# Esta função permite executar os cálculos de features de Lipinski e PubChem Fingerprints

### ***2. Calcular descritores de Lipinski e PubChemFP para todos SMILES***

In [ ]:
# Nomes das colunas
colunas_descritores = ['MolLogP', 'MolWt', 'nHBDon', 'nHBAcc']
colunas_fingerprints = [f'PubChemFP_{i}' for i in range(881)]

# Calcular para todos os SMILES
dados = df['SMILES'].apply(calcular_descritores_fp)

df_descritores_fp = pd.DataFrame(dados.tolist(), columns=colunas_descritores + colunas_fingerprints)
# Executa o cálculo das features e insere os resultados na DataFrame

### ***3. Concatenar com dados pré-processados***

In [ ]:
# Juntar com informações originais
df_final = pd.concat([df.reset_index(drop=True), df_descritores_fp.reset_index(drop=True)], axis=1)

# Ver resultado
print(df_final.shape)
df_final.head()

(2694, 893)


,ChEMBL_ID,SMILES,IC50_nM,Mol,Canonical_SMILES,pIC50,Classe,Name,MolLogP,MolWt,...,PubChemFP_871,PubChemFP_872,PubChemFP_873,PubChemFP_874,PubChemFP_875,PubChemFP_876,PubChemFP_877,PubChemFP_878,PubChemFP_879,PubChemFP_880
0,CHEMBL68920,Cc1cc(C)c(/C=C2\C(=O)Nc3ncnc(Nc4ccc(F)c(Cl)c4)...,300.0,<rdkit.Chem.rdchem.Mol object at 0x78a581d71a10>,C=C1C=C(C)NC1=CC1=C(O)NC2=NC=NC(NC3=CC=C(F)C(C...,6.522879,1,0,4.45034,383.814,...,1,1,1,0,1,1,1,1,1,1
1,CHEMBL69960,Cc1cc(C(=O)N2CCOCC2)[nH]c1/C=C1\C(=O)Nc2ncnc(N...,400.0,<rdkit.Chem.rdchem.Mol object at 0x78a581d718c0>,C=C1C=C(C(=O)N2CCOCC2)NC1=CC1=C(O)NC2=NC=NC(NC...,6.397940,1,1,3.61432,482.903,...,1,1,1,0,1,1,1,1,1,1
2,CHEMBL67057,Cc1cc(C(=O)N2CCOCC2)[nH]c1/C=C1\C(=O)Nc2ncnc(N...,100.0,<rdkit.Chem.rdchem.Mol object at 0x78a581d71930>,C=C1C=C(C(=O)N2CCOCC2)NC1=CC1=C(O)NC2=NC=NC(NC...,7.000000,1,2,4.82482,559.630,...,1,1,1,1,1,1,1,1,1,1
3,CHEMBL65848,Cc1cc(C(=O)N2CCOCC2)[nH]c1/C=C1\C(=O)Nc2ncnc(N...,5000.0,<rdkit.Chem.rdchem.Mol object at 0x78a581d71850>,C=C1C=C(C(=O)N2CCOCC2)NC1=CC1=C(O)NC2=NC=NC(NC...,5.301030,0,3,3.31052,470.533,...,1,1,1,0,1,1,1,1,1,1
4,CHEMBL69629,Cc1cc(C(=O)NCCN2CCOCC2)[nH]c1/C=C1\C(=O)Nc2ncn...,100.0,<rdkit.Chem.rdchem.Mol object at 0x78a581d717e0>,C=C1C=C(C(=O)NCCN2CCOCC2)NC1=CC1=C(O)NC2=NC=NC...,7.000000,1,4,3.20392,525.972,...,1,1,1,0,1,1,1,1,1,1


### ***4. Salvar os dados para EDA***

In [ ]:
df_final.to_csv('Dados para EDA.csv', index=False, encoding='utf-8')

### ***FIM***